In [1]:
#==========簡易香水網站(1976)爬蟲程式==========
import requests
from bs4 import BeautifulSoup
import xlwings as xw
import time

#定義一個函數擷取香水商品網頁的規格及價錢
def perfume_crawler(perfume_no):
    res = requests.get(f"https://www.1976.com.tw/prod/{perfume_no}") #變數為香水網頁代號
    html_perfume = BeautifulSoup(res.text, "html.parser")
    table_perfume = html_perfume.findAll("table", {"width" : "750px"})[0] #擷取產品表格 (由於table只有一個, 必須寫上索引值)
    product = table_perfume.findAll("td", {"class" : "product-td1"}) #擷取商品名稱
    td_ml_ori_price = table_perfume.findAll("td", {"class" : "product-td2"}) #擷取規格及市價
    span_price = table_perfume.findAll("span", {"class" : "product-price"}) #擷取特價
    #計算商品有幾種規格
    product_count = len(product)
    #將商品名稱整理成字串存成list
    product_name = []
    for i in range(0, product_count):
        product_name.append(product[i].text.replace("\n", ""))
    #由於規格及市價被放在同個td及class下, 將規格及市價拆開 (list長度為商品規格的兩倍)
    #擷取規格 (第奇數個)
    ml = [td_ml_ori_price[i].text for i in range(0, product_count * 2) if i % 2 == 0 ]
    #擷取市價 (第偶數個), 並將市價中的逗點去除
    ori_price = [td_ml_ori_price[i].text.replace(",", "") for i in range(0, product_count * 2) if i % 2 != 0 ]
    #將特價中的逗點去除
    price = [span.text.replace(",", "") for span in span_price]
    #將商品名稱, 規格, 市價, 特價資訊回傳成一個字典
    return {
        "product_name" : product_name,
        "ml" : ml,
        "ori_price" : ori_price,
        "price" : price
    }
#函數定義結束

#將網頁爬蟲完的資料存入excel檔案
wb = xw.Book("perfume_price_1976.xlsx") #呼叫存有每日香水價格的excel檔案
perfume_no = ["7869", "4869", "3182"] #輸入關注的香水網頁代號
#將香水對應的商品名稱, 規格, 市價, 特價以迴圈方式輸入至對應的工作表內
for perfume in perfume_no:
    #選至對應的香水代號工作表
    sheet = wb.sheets[f"{perfume}"]
    #擷取香水資料
    perfume_data = perfume_crawler(perfume)
    #確認商品名稱有幾種 (需要輸入幾列, 下面新增資料的迴圈會使用到)
    product_count = len(perfume_data["product_name"])
    #抓到該工作表內的最後一列
    last_row = sheet.range("A1").end("down").row
    #從最後一列以後開始輸入資料
    for i in range(0, product_count):
        sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d") #記錄擷取資料的日期
        sheet.range(f"B{last_row+1}").value = perfume_data["product_name"][i]
        sheet.range(f"C{last_row+1}").value = perfume_data["ml"][i]
        sheet.range(f"D{last_row+1}").value = perfume_data["ori_price"][i]
        sheet.range(f"E{last_row+1}").value = perfume_data["price"][i]
        last_row = last_row + 1
#存入完成